### Define helper functionality to visualize forward and inverse image mapping

In [ ]:
import os

from pathlib import Path
from typing import Callable

# import Metashape
import numpy as np

from mynd.backend import metashape as backend
from mynd.camera import CameraCalibration
from mynd.collections import CameraGroup, StereoCameraGroup
from mynd.geometry import HitnetModel, load_hitnet
from mynd.image import Image, filter_image_clahe

from mynd.tasks.export_stereo import ExportStereoTask, invoke_stereo_export_task

from mynd.utils.environment import Environment, load_environment
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


GroupID = CameraGroup.Identifier


def create_clahe_filter(clip: float, size: int) -> Callable:
    """Creates a CLAHE filter with the given parameters."""

    def clahe_filter(image: Image) -> Image:
        return filter_image_clahe(image, clip=clip, size=size)

    return clahe_filter


def filter_disparity_map(disparity: np.ndarray) -> np.ndarray:
    """Filter a disparity map."""

    # TODO: Add disparity filter

    return disparity


def log_camera_calibration(calibration: CameraCalibration) -> None:
    """Logs a camera calibration."""
    # TODO: Change numpy precision output
    with np.printoptions(precision=4, suppress=True, threshold=5):
        logger.info(f"Camera matrix:  {calibration.camera_matrix}")
        logger.info(f"Distortion:     {calibration.distortion}")
        logger.info(f"Location:       {calibration.location}")
        logger.info(f"Rotation:       {calibration.rotation}")


def prepare_task_config(
    identifier: GroupID,
    stereo_camera: StereoCameraGroup,
    range_directory: Path,
    normal_directory: Path,
) -> ExportStereoTask.Config:
    """Prepares a stereo export task config by creating setting up paths, creating directories,
    and loading a disparity model."""

    environment: Environment = load_environment().unwrap()

    disparity_estimator: HitnetModel = load_hitnet(
        environment.resource_directory
        / Path("hitnet_models/hitnet_eth3d_480x640.onnx")
    ).unwrap()

    # NOTE: Debug only!
    logger.info(
        "-------------------- Camera calibration 1 --------------------"
    )
    log_camera_calibration(stereo_camera.calibrations.first)
    logger.info(
        "-------------------- Camera calibration 2 --------------------"
    )
    log_camera_calibration(stereo_camera.calibrations.second)
    logger.info("")

    logger.info(f"Camera pairs: {len(stereo_camera.camera_pairs)}")
    logger.info(f"Image loaders: {len(stereo_camera.image_loaders)}")

    # TODO: Tune
    clahe_filter = create_clahe_filter(clip=10.0, size=20)

    config = ExportStereoTask.Config(
        range_directory=range_directory / Path(f"{identifier.label}_ranges"),
        normal_directory=normal_directory / Path(f"{identifier.label}_normals"),
        calibrations=stereo_camera.calibrations,
        camera_pairs=stereo_camera.camera_pairs,
        image_loaders=stereo_camera.image_loaders,
        # Processors
        disparity_estimator=disparity_estimator,
        image_filter=None,
        disparity_filter=None,
    )

    # Create export directories if they do not exist
    if not config.range_directory.exists():
        os.mkdir(config.range_directory)
    if not config.normal_directory.exists():
        os.mkdir(config.normal_directory)

    return config


def batch_stereo_export(
    target: GroupID,
    stereo_groups: list[StereoCameraGroup],
    range_directory: Path,
    normal_directory: Path,
) -> None:
    """Iterate over the stereo groups in a camera group."""
    for stereo_group in stereo_groups:

        config: ExportStereoTask.Config = prepare_task_config(
            target, stereo_group, range_directory, normal_directory
        )

        result: Result[ExportStereoTask.Result, str] = (
            invoke_stereo_export_task(config)
        )


def main():
    """Main function."""

    DOCUMENT_PATH: Path = Path(
        "/data/kingston_snv_01/acfr_metashape_projects_dev/r29mrd5h_lite_with_metadata.psz"
    )

    RANGE_DIRECTORY: Path = Path("/data/kingston_snv_01/stereo_test/ranges")
    NORMAL_DIRECTORY: Path = Path("/data/kingston_snv_01/stereo_test/normals")

    DEPLOYMENT: str = "r29mrd5h_20090612_225306"

    # r29mrd5h_20090612_225306, r29mrd5h_20090613_100254, r29mrd5h_20110612_033752, r29mrd5h_20130611_002419
    # qdch0ftq_20100428_020202, qdch0ftq_20110415_020103, qdch0ftq_20120430_002423
    # r23685bc_20100605_021022, r23685bc_20120530_233021, r23685bc_20140616_225022
    # r23m7ms0_20100606_001908, r23m7ms0_20120601_070118, r23m7ms0_20140616_044549

    match backend.load_project(DOCUMENT_PATH):
        case Ok(None):
            pass
        case Err(message):
            logger.error(message)

    groups: dict[str, GroupID] = {
        group.label: group for group in backend.get_group_identifiers().unwrap()
    }

    target: GroupID = groups.get(DEPLOYMENT)

    logger.info(f"Target deployment: {target}")

    match backend.camera_services.retrieve_stereo_cameras(target):
        case Ok(stereo_camera_groups):
            batch_stereo_export(
                target,
                stereo_camera_groups,
                range_directory=RANGE_DIRECTORY,
                normal_directory=NORMAL_DIRECTORY,
            )
        case Err(message):
            logger.error(message)


# ---------- Invoke main function ----------
main()